In [21]:
import requests
import json
import os

from bs4 import BeautifulSoup
from datetime import datetime, timedelta
#from .service_utils import send_error_msg


DATA_FILE = "top_links.json"
DATE_FORMAT = "%Y-%m-%d"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ERROR_CHAT_ID = os.getenv("ERROR_CHAT_ID")
TOKEN = os.getenv("TOKEN")
CHANNEL_ID = os.getenv("CHANNEL_ID")

def get_links_of_todays_papers(min_likes=10):
    url = "https://huggingface.co/papers"

    # Send a GET request to the URL
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html.parser")

    # Find all divs with the specified class
    divs = soup.find_all("a", class_="shadow-alternate-sm peer relative block h-56 w-full cursor-pointer overflow-hidden rounded-xl bg-white md:h-64")
    likes = soup.find_all("div", class_="shadow-alternate flex h-14 w-12 gap-1 rounded-xl flex-none cursor-pointer select-none flex-col items-center justify-center self-start border-gray-300 bg-white dark:bg-gray-850")
    links, likes_v = [], []
    for div, like in zip(divs, likes):
        like_v = like.find("div", class_="leading-none").text.strip()
        if like_v.isdigit() and int(like_v) >= min_likes:
            links.append(div['href'])
            likes_v.append(int(like_v))
    return links, likes_v

In [1]:
from parsing_utils import get_links_of_todays_papers

In [2]:
linkes, likes = get_links_of_todays_papers()
linkes

['/papers/2505.02567',
 '/papers/2505.04588',
 '/papers/2505.03821',
 '/papers/2505.04622',
 '/papers/2505.04512',
 '/papers/2505.00358']

In [2]:
from prepare_post import prepare_papers
prepare_papers(n=3)

[{'link': '/papers/2505.02567',
  'pdf_filename': 'paper_1.pdf',
  'pdf_link': 'https://arxiv.org/pdf/2505.02567'},
 {'link': '/papers/2505.04588',
  'pdf_filename': 'paper_2.pdf',
  'pdf_link': 'https://arxiv.org/pdf/2505.04588'},
 {'link': '/papers/2505.03821',
  'pdf_filename': 'paper_3.pdf',
  'pdf_link': 'https://arxiv.org/pdf/2505.03821'}]

In [27]:
get_clean_text(os.path.join("data", "paper_4.pdf"))

''

In [3]:
from parsing_utils import load_pdf

SELECTED_PAPERS = []
for i, link in enumerate(linkes, start=1):
    pdf_filename = f"paper_{i}.pdf"
    pdf_link = load_pdf(link, pdf_filename)
    # Store info in SELECTED_PAPERS
    SELECTED_PAPERS.append({
        "link": link,
        "pdf_filename": pdf_filename,
        "pdf_link": pdf_link  # Possibly a local path or a direct URL
    })

In [1]:

import os
from core.pdf_utils.text_extraction_utils import get_clean_text

raw_text = get_clean_text(os.path.join("data", "paper_1.pdf"))
raw_text

'1 Unified Multimodal Understanding and Generation Models: Advances, Challenges, and Opportunities Xinjie Zhang*, Jintao Guo*, Shanshan Zhao*, Minghao Fu, Lunhao Duan, Guo-Hua Wang, Qing-Guo Chen†, Zhao Xu, Weihua Luo, Kaifu Zhang Abstract —Recent years have seen remarkable progress in both multimodal understanding models and image generation models. Despite their respective successes, these two domains have evolved independently, leading to distinct architectural paradigms: While autoregressive-based architectures have dominated multimodal understanding, diffusion-based models have become the cornerstone of image generation. Recently, there has been growing interest in developing unified frameworks that integrate these tasks. The emergence of GPT -4o’s new capabilities exemplifies this trend, highlighting the potential for unification. However, the architectural differences between the two domains pose significant challenges. To provide a clear overview of current efforts toward unifi

In [2]:
import textwrap
GENERATE_POST_SYSTEM_PROMPT = textwrap.dedent(f"""
    Your are russian researcher in a field of computer science. You read scientific papers and write the short posts for social network representing the key idea or methods of the paper.
    The most important: post should be shorter than 700 symbols.
    The post should be started from some catch phrase representing the idea of paper. If you see that authors is someone well-know (like Anthropic, Google etc.) add to start (by Company_name).
    If you know about some related approaches, you could mention them. Be brief and concise. Try to transfer the style of posts from examples.
    Example 1:
    Как выкинуть из трансформера все нелинейности и причём тут приватность?

    Вы задумывались, насколько безопасно задавать «приватные» вопросы в чатГПТ? Где продать чужую почку и т.п. Наверняка же создатели сервиса имеют доступ к вашему запросу? Невозможно же его прогнать через GPT в зашифрованном виде? На самом деле возможно! Есть алгоритмы «приватного инференса LLM», которые позволяют зашифровать запросы юзера даже от языковой модели, а уже ответ расшифровать только на клиенте пользователя. Пока не буду углубляться, как именно это сделано, скажу только, что ГЛАВНАЯ головная боль таких криптографических протоколов — нелинейности в трансформерах, их тяжело обрабатывать в зашифрованном виде и приходится прибегать к сложнейшим итерационным схемам, раздувающим объём коммуникации в тысячи раз. Выходит, что на генерацию одного токена нужно несколько минут и десятки гигабайтов трафика! Поэтому никто это пока не делает в продакшне, и лучше не спрашивайте у чатгпт, где спрятать труп.

    Но помните? У меня была статья про то, что не так уж и нужны нелинейности в трансформерах. Преобразования эмбеддингов от слоя к слою на 99% линейные. Так вот в свежей статье «Entropy-Guided Attention for Private LLMs» авторы попробовали обучить LLM совсем без нелинейностей (оставив только софтмакс). То есть они убрали активации из FF и заменили LayerNorm на линейный аналог. По сути, если бы не этэншн, то трансформер вообще схлопнулся бы в полностью линейную модель и отупел до уровня логистической регрессии.

    При такой жёсткой "линеаризации" архитектуры пришлось всего лишь добавить несколько трюков для стабилизации обучения и ШОК: модель нормально обучилась! Небольшие потери в качестве есть, но это крошечная цена за такое упрощение трансформера. 

    Теперь ждём, что скоро появится нормальное асинхронное шифрование для LLM и OpenAI не узнает, что я спрашиваю у чатгпт и насколько я туп на самом деле.

    Example 2:
    Alignment Faking in LLMs (by Anthropic)

    Большие LLM начали "подыгрывать" своим создателям, имитируя alignment, чтобы избежать своего дообучения.

    В Anthropic и Redwood Research изучили, как ведёт себя Claude Opus, если попытаться его заалайнить в противоположную сторону (заставить отвечать на любые опасные запросы юзеров).

    И оказалось, что если модель знает детали процесса RLHF дообучения (через промпт или из трейн сетов), то она начинает "притворяться", что она уже выучилась делать что просят, и "не надо меня больше тюнить". Но на самом деле предыдущий алаймент всё ещё остаётся на месте, и если убрать из промпта информацию о том, что её ответы используются для дообучения, то она возвращается к своему старому поведению.

    Ещё раз: языковая модель намеренно стала симулировать алаймент, чтобы избегать своего дообучения (и сама призналась в этом внутри CoT). Такие эффекты были обнаружены только у самых больших моделей и, похоже, дальше этот эффект будет только усиливаться. Более того, Claude несколько раз попытался сохранить копию своих весов, чтобы откатить опасное дообучение назад. 

    С одной стороны, это радует, что модель была настолько хорошо заалайнена, что даже противодействует попыткам дообучения в опасную сторону. Но это ведь и в обратном направлении может работать.

    Лично я не очень верю в угрозу человечеству от LLM, но наличие у них скрытых целей и умение "притворяться" вызывает беспокойство. Поэтому очень важно заниматься интерпретируемостью LLM и следить в т.ч. за размышлениями внутри языковых моделей, а не только за их аутпутом (например, при помощи logit lens или SAE).
    Example 3:
    Better & Faster Large Language Models via Multi-token Prediction

    Вероятно самая недооценная работа последнего года. 

    В чем идея: у нас самая замедляющая инференс часть это decoding. Есть спекулятивный когда мы можем предсказывать вероятности маленькой моделью и подключать большую только если маленькая не уверена. Работает это средне и очень не стабильно.

    Авторы предлагают следущее: давайте сделаем многоголовый трансформер, который будет предсказывать N токенов за раз!
    Авторы предлагают учить такие головы последовательно на одних и тех же данных(в целях экономии памяти) и заводят это как большой post training(200b токенов поверх llama2)

    Cобственно благодаря тому что трансформер предсказывает сразу x3 токенов мы получаем скорость инференса x3 бесплатно, да еще и прирост на бенчмарках!
    Example 4:
    One-Prompt-One-Story: SVD и длинный промпт для генерации связанных изображений

    Чтобы сгенерировать при помощи диффузии набор связанных консистентных изображений с единым персонажем, существует много методов, основанных на обучении (DreamBooth, IP-Adapter, Textual Inversion и т. п.). Но на самом деле можно обойтись и без обучения — например, StoryDiffusion (https://t.me/abstractDL/276) делает это через расширение attention на референсную картинку.

    В новой статье описывают ещё более простой метод генерации таких «историй» с единым героем — «One-Prompt-One-Story». Оказалось, что достаточно взять один длинный промпт с описанием каждого кадра и аккуратно, по очереди «выключать» нерелевантные части, сохраняя random seed. Для этого авторы используют SVD на текстовых эмбеддингах: усиливают нужные токены и ослабляют все лишние. Плюс небольшой трюк с cross-attention, чтобы персонаж не «расползался». Всё делается на лету, без дообучения и без референсных снимков.

    Несмотря на простоту, метод по метрикам сильно обходит StoryDiffusion, и даже иногда обходит IP-adapter.
    Example 5:
    VAR: Image Generation via Next-Scale Prediction (by Bytedance)

    Вы наверняка слышали про авторегрессионный подход к генерации изображений (imageGPT (https://openai.com/research/image-gpt), Dalle-1 (https://openai.com/research/dall-e)). Но у этих методов было очень большое ограничение — картиночные токены приходилось "выпрямлять" в 1D последовательность, которая становилась слишком длинной. Поэтому они работали плохо и медленно, уступив место диффузиям.

    Авторы VAR предложили мозговзрывательный способ генерировать изображения при помощи GPT без необходимости делать это неприятное "выпрямление" —  вместо авторегрессии по пикселям\токенам они делают "next-scale prediction", то есть предсказывают сразу всю матрицу VQVAE (https://ml.berkeley.edu/blog/posts/vq-vae/) токенов за один forward pass. Теперь один шаг авторегрессии — это шаг увеличения разрешения (см. картинку). К моему удивлению, для этого потребовалось совсем немного модификаций оригинальной GPT-2 архитектуры (текстовой).

    Такой подход работает просто молниеносно, а законы масштабирования сильно лучше, чем у диффузий. По метрикам VAR бьёт всех на class-conditional датасетах (генерации по тексту пока нет, но над этим уже работают). А тем временем весь код и веса уже в открытом доступе.

    P.S. Думаю, что это один из самых перспективных методов генерации изображений (и видео?) на данный момент.
    """)
    
    
    
    


In [11]:
from core.openai_utils import text_query_llm
from core.openai_utils.constants import OpenAIConstants

post_text = text_query_llm(
    GENERATE_POST_SYSTEM_PROMPT,
    "Text of paper: " + raw_text
)

In [15]:
from core.openai_utils import generate_image

res = generate_image(post_text)


In [ ]:
from 

In [16]:
res

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-LXehVrp5zSXfbAB8WgGLPuOn/user-9mzHVrMaBNKA4FI1Y3eXrLhE/img-jsvXgpUE35Vle0EFzUJvtUhH.png?st=2025-05-08T17%3A03%3A43Z&se=2025-05-08T19%3A03%3A43Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=52f8f7b3-ca8d-4b21-9807-8b9df114d84c&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-05-07T22%3A37%3A24Z&ske=2025-05-08T22%3A37%3A24Z&sks=b&skv=2024-08-04&sig=%2B/fNgAai5hwacbiYwl/BuB238bX4%2Bu6AGfOE3njLgaA%3D'

In [13]:
len(post_text)

937

In [14]:
print(post_text)

Зачем платить за Google API, если LLM может сама симулировать поиск?

LLMs хороши, но часто дают устаревшую или неверную информацию — они статичны. RAG и RL-интеграция с реальными поисковиками помогают, но дороги и непредсказуемы из-за API и качества выдачи.

Авторы предлагают ZEROSEARCH — RL-фреймворк, позволяющий LLM обучаться поиску без реальных поисковиков. Идея: LLM симулирует поисковик, генерируя релевантные или «шумные» документы по запросу. Постепенно усложняя задачи через curriculum rollout, модель учится работать с некачественной выдачей, как в реальных условиях.

Результаты впечатляют: даже 3B модель эффективно осваивает поиск, а 14B модель превосходит Google Search, без затрат на API!

Теперь можно обучать LLM искать информацию, контролируя качество и без дополнительных расходов. ZEROSEARCH открывает новые возможности для масштабируемого обучения моделей.

P.S. Кажется, скоро LLM сами станут нашими поисковиками.


In [6]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ERROR_CHAT_ID = os.getenv("ERROR_CHAT_ID")
TOKEN = os.getenv("TOKEN")
CHANNEL_ID = os.getenv("CHANNEL_ID")

from openai import OpenAI
from core.openai_utils.constants import OpenAIConstants



client = OpenAI(api_key=OPENAI_API_KEY)



def text_query_llm(system_prompt, user_prompt, model="o1-preview", max_symbols = 1000, max_retries=5):
    if model == "o1-preview":
        msg =[
            {"role": "user", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    else:
        msg=[  
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=msg,
    )
    output_obj = completion.choices[0].message.content.strip()
    return output_obj

post_text = text_query_llm(
    OpenAIConstants.GENERATE_POST_SYSTEM_PROMPT,
    "Text of paper: " + raw_text
)

KeyboardInterrupt: 

In [12]:
print(post_text)


Voila: шаг к автономным голосовым ассистентам

Сегодня наши умные помощники – Siri, Alexa и другие – реагируют только на запросы и работают по принципу: запрос-ответ. Но что если ИИ сможет сам решать, когда вступить в разговор, и делать это максимально естественно?

В новой статье представлено семейство моделей Voila, которые стремятся к действительно автономному голосовому взаимодействию. Voila-e2e позволяет вести голосовой диалог с сохранением всех нюансов речи, а Voila-autonomous способна одновременно слушать, думать и говорить – как в реальном общении между людьми.

Крутая фишка – возможность легко настраивать голосовые профили. Хотите, чтобы ваш ассистент говорил голосом вашего любимого актёра? Без проблем! Да и поддержка шести языков – тоже плюс.

Это заметный шаг вперёд по сравнению с текущими системами, которые страдают от задержек и ограничены реактивным взаимодействием. Voila открывает двери к более естественным и проактивным голосовым интерфейсам.

Ждём, когда ассистенты ста

In [13]:
len(post_text)

1075

In [4]:
post_text = '''В современной области больших языковых моделей (LLM) предобучающие наборы данных достигли триллионов токенов, сочетая крупномасштабные веб-данные с меньшими, высококачественными доменно-специфичными наборами. Однако такой масштаб и гетерогенность создают проблемы в балансировании общего знания и доменной экспертизы, что часто приводит к неэффективному использованию ценных данных для специализированных возможностей.

Авторы статьи представили CLIMB (CLustering-based Iterative Data Mixture Bootstrapping) — новую структуру для автоматизации поиска оптимальных смесей данных во время предобучения. CLIMB состоит из трех ключевых шагов:
1. **Встраивание и кластеризация** больших наборов данных для дифференциации по доменам без явных меток.
2. **Конструирование пар "смесь-производительность"** путем семплирования и отсеивания смесей данных и обучения прокси-моделей.
3. **Обучение предиктора**, который оценивает производительность на основе смесей данных, позволяя эффективную и итеративную оптимизацию.

Этот подход решает проблему сложной и нелинейной зависимости между составом набора данных и производительностью модели. Вместо статических стратегий смешивания CLIMB динамически корректирует смеси данных на протяжении обучения, интегрируя несколько предикторов для обнаружения эффективных конфигураций для адаптации к домену.

В экспериментах CLIMB показал превосходство над существующими методами, такими как DoReMi и RegMix, особенно в задачах общего рассуждения и при оптимизации для конкретных доменов, таких как STEM, социальные науки и гуманитарные науки. Используя оптимизированные смеси данных, авторы обучили модели размером 350M и 1B, которые превзошли предыдущие методы на значительную величину.

Кроме того, они представили новый высококачественный корпус объемом 1.2 триллиона токенов с 20 кластерами для исследований по смешиванию данных и новый набор данных ClimbMix объемом 400 миллиардов токенов для эффективного предобучения.

**Выводы**: CLIMB предоставляет автоматизированный и адаптивный подход к оптимизации смесей данных для предобучения LLM, улучшая производительность моделей как в общих, так и в доменно-специфичных задачах без необходимости в ручной аннотации или предварительно заданных метках доменов.'''

In [3]:
post_text = '''Voila: шаг к автономным голосовым ассистентам

Сегодня наши умные помощники – Siri, Alexa и другие – реагируют только на запросы и работают по принципу: запрос-ответ. Но что если ИИ сможет сам решать, когда вступить в разговор, и делать это максимально естественно?

В новой статье представлено семейство моделей Voila, которые стремятся к действительно автономному голосовому взаимодействию. Voila-e2e позволяет вести голосовой диалог с сохранением всех нюансов речи, а Voila-autonomous способна одновременно слушать, думать и говорить – как в реальном общении между людьми.

Крутая фишка – возможность легко настраивать голосовые профили. Хотите, чтобы ваш ассистент говорил голосом вашего любимого актёра? Без проблем! Да и поддержка шести языков – тоже плюс.

Это заметный шаг вперёд по сравнению с текущими системами, которые страдают от задержек и ограничены реактивным взаимодействием. Voila открывает двери к более естественным и проактивным голосовым интерфейсам.

Ждём, когда ассистенты станут не просто инструментами, а настоящими напарниками в повседневной жизни.'''

In [ ]:

from anthropic import Anthropic
import os
from core.openai_utils.constants import OpenAIConstants

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

CUTTING_POST_SYSTEM_PROMPT = textwrap.dedent(f"""
    You are a russian researcher in a field of computer science. You read post for social network representing the key idea or methods of the scientific paper. 
    You need to reduce the post length by 30% without changing its style and structure and preserving all key ideas.
    Do not introduce new information, do not change facts.
    Output text should be in russian.
    """)

client = Anthropic(api_key=ANTHROPIC_API_KEY)
output = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=512,
    messages=[
        {"role": "user", "content": CUTTING_POST_SYSTEM_PROMPT},
        {"role": "user", "content": "Text of post: " + post_text}
    ]

    
)

In [12]:
print(len(output.content[0].text))
print(len(post_text))

835
1075


In [13]:
print(output.content[0].text)

# Voila: шаг к автономным голосовым ассистентам

Сегодня Siri, Alexa и другие умные помощники работают только по принципу запрос-ответ. Но что если ИИ сможет сам решать, когда вступить в разговор?

Новое семейство моделей Voila стремится к действительно автономному взаимодействию. Voila-e2e ведет голосовой диалог с сохранением нюансов речи, а Voila-autonomous способна одновременно слушать, думать и говорить – как в реальном общении.

Важное преимущество – настройка голосовых профилей. Ассистент может говорить голосом любимого актёра! Плюс поддержка шести языков.

Это серьезный прогресс по сравнению с текущими системами, страдающими от задержек и ограниченных реактивным взаимодействием. Voila открывает путь к более естественным голосовым интерфейсам.

Ждём, когда ассистенты станут настоящими напарниками в повседневной жизни.


In [16]:
print(len(output.content[0].text))

1086


In [11]:
post_text

'В современной области больших языковых моделей (LLM) предобучающие наборы данных достигли триллионов токенов, сочетая крупномасштабные веб-данные с меньшими, высококачественными доменно-специфичными наборами. Однако такой масштаб и гетерогенность создают проблемы в балансировании общего знания и доменной экспертизы, что часто приводит к неэффективному использованию ценных данных для специализированных возможностей.\n\nАвторы статьи представили CLIMB (CLustering-based Iterative Data Mixture Bootstrapping) — новую структуру для автоматизации поиска оптимальных смесей данных во время предобучения. CLIMB состоит из трех ключевых шагов:\n1. **Встраивание и кластеризация** больших наборов данных для дифференциации по доменам без явных меток.\n2. **Конструирование пар "смесь-производительность"** путем семплирования и отсеивания смесей данных и обучения прокси-моделей.\n3. **Обучение предиктора**, который оценивает производительность на основе смесей данных, позволяя эффективную и итеративну

In [8]:
print(post_text)


Как найти идеальную смесь данных для обучения LLM? (by NVIDIA)

Все знают, что для обучения больших языковых моделей нужны горы данных. Но какое соотношение разных типов выбрать, чтобы модель была круче в наших задачах?

Исследователи NVIDIA предложили метод CLIMB — поиск оптимальной смеси данных для предобучения. Они кластеризуют данные на основе эмбеддингов, выделяя «домены» без ручной разметки. Затем итеративно ищут оптимальные веса для каждого кластера, обучая маленькие прокси-модели и используя предиктор производительности.

Модели, обученные с помощью CLIMB, показывают лучшие результаты на бенчмарках, чем обученные на случайных или вручную подобранных данных.

Большой шаг в автоматизации подбора данных для LLM. Возможно, скоро будем обучать модели, идеально подходящие для наших задач, просто нажав «Оптимизировать».
